# Multy-Domain Sentiment Dataset: Sentiment Analisys per domain

## 1. Preprocesamiento
Para el preprocesamiento, se creó la clase "Corpus" que recibe una serie de documentos y obtiene las etiquetas de cada uno, el vocabulario, y las representaciones tf y tfidf.

In [26]:
import os
import pandas as pd
import numpy as np
import math

POSITIVE_LABEL = 1
NEGATIVE_LABEL = 0

CATEGORY_PATHS = {
    "books": "../data/books/",
    "dvd": "../data/dvd/",
    "electronics": "../data/electronics/",
    "kitchen": "../data/kitchen/",
}


class Corpus:
    # Atributes
    __vocabulary = {}
    __tf = None
    __tfidf = None
    __documents = None


    def __init__(self, docs: list, vocabulary: dict = None) -> None:
        """
        Creates the vocabulary, the TF and the TFIDF matricees from docs.
        
            Params
            ------
                docs: pd.DataFrame
                    List of documents.
                
                vocabulary: dict | None (default: None)
                    A dictionary where the keys are the terms in the vocabulary
                    and the values are each term's unique id. If set to None,
                    this class will create the vocabulary based on docs.
                    WARNING: if specified, vocabulary must contain a "#UNK#" key
                    for unknown terms.
        """        
        self.__documents = self.__get_docs_df(docs)
        if vocabulary is None:
            self.__vocabulary = self.__load_vocabulary()
        else:
            self.__vocabulary = vocabulary
            
        self.__tf = self.__load_tf()
        self.__tfidf = self.__load_tfidf()

    def __get_docs_df(self, docs: list):
        documents = []
        for doc in docs:
            terms = self.__get_term_counts(doc)
            label = self.__get_label(doc)
            documents.append([terms, label])
        
        return pd.DataFrame(documents, columns=["terms", "label"])

    def __get_term_counts(self, line: str) -> list:
        line_arr = line.split()
        line_arr = line_arr[:-1] # Remove #label#: from the end of array
        terms = {}
        for term in line_arr:
            term_arr = term.split(":")
            terms[term_arr[0]] = int(term_arr[1])
        
        return terms
    
    def __get_label(self, doc: str) -> int:
        doc_arr = doc.split()
        label_str = doc_arr[-1]
        label = label_str.split(":")[-1]
        if label.lower() == "negative":
            return NEGATIVE_LABEL
        elif label.lower() == "positive":
            return POSITIVE_LABEL
    
    def __load_vocabulary(self):
        # Term counts in the whole corpus
        voc = {}
        docs = self.__documents
        for i in range(len(docs)):
            terms_dict = docs.loc[i, "terms"]
            for term in terms_dict:
                if term in voc:
                    voc[term] += terms_dict[term]
                else:
                    voc[term] = terms_dict[term]
        
        # Replace terms with one appearance with UNK
        terms_to_del = []
        for term in voc:
            if voc[term] == 1:
                terms_to_del.append(term)

        # Remove terms that appeare only once
        for term in terms_to_del:
            voc.pop(term)
        
        # Assign unique ids to terms in vocabulary
        bow = {"#UNK#": 0}
        i = 1
        for term in voc:
            bow[term] = i
            i += 1
        
        return bow

    def __load_tf(self) -> np.ndarray:
        voc = self.__vocabulary
        docs = self.__documents
        tf = np.zeros((len(docs), len(voc)))

        for i in range(len(docs)):
            doc = docs.iloc[i]
            for term in doc["terms"]:
                if term in voc:
                    tf[i][voc[term]] = doc["terms"][term]
                else:
                    tf[i][voc["#UNK#"]] += doc["terms"][term]
        
        return tf

    def __load_tfidf(self) -> np.ndarray:
        voc = self.__vocabulary
        docs = self.__documents
        tfs = self.__tf
        doc_frec = np.count_nonzero(tfs, axis=0)
        total_docs = len(docs)
        tfidf = np.zeros((len(docs), len(voc)))
        for i in range(len(docs)):
            doc = docs.iloc[i]
            unk_tf = 0
            for term in doc["terms"]:
                if term in voc:
                    term_id = voc[term]
                    tf = doc["terms"][term]
                    df = int(doc_frec[term_id])
                    tfidf[i][term_id] = math.log10(1+tf) * math.log10(total_docs / df)
                else:
                    unk_tf += doc["terms"][term]
            
            # Calculate UNK tfidf
            term_id = voc["#UNK#"]
            tf = int(tfs[i][term_id])
            df = int(doc_frec[term_id])
            tfidf[i][term_id] = math.log10(1+tf) * math.log10(total_docs / df)
        
        return tfidf

    def getVocabulary(self) -> dict:
        return self.__vocabulary
    
    def getDocuments(self) -> pd.DataFrame:
        return self.__documents

    def getTfs(self) -> np.ndarray:
        return self.__tf
    
    def getTfidfs(self) -> np.ndarray:
        return self.__tfidf

# 2. Construcción del modelo y métricas por categoría

In [27]:
def calculate_metrics(labels: list, predicted_labels: list) -> dict:
    tp = 0
    fp = 0
    fn = 0
    tn = 0

    for i in range(len(labels)):
        act     = labels[i]
        pred    = predicted_labels[i]
        if(act == 1 and pred == 1):
            tp += 1
        elif pred == 1 and act == 0:
            fp += 1
        elif pred == 0 and act == 1:
            fn += 1
        elif pred == 0 and act == 0:
            tn += 1
    
    presition = tp / (tp + fp)
    recall = tp / (tp + fn)
    f1 =  2 * presition * recall / (presition + recall)

    return {"presition": presition, "recall": recall, "f1": f1}
    


In [28]:
from sklearn.naive_bayes import MultinomialNB

def test_nv_classifier(X_train: np.ndarray, y_train, X_test: np.ndarray, y_test) -> dict:
    model = MultinomialNB()
    model.fit(X_train, y_train)
    y_predicted = model.predict(X_test)
    return calculate_metrics(y_test, y_predicted)

In [29]:
import os

def get_docs_from_folder_paths(folder_paths: list):
    train_docs = []
    test_docs = []
    for path in folder_paths:
        positive_path = os.path.join(path, "positive.review")
        negative_path = os.path.join(path, "negative.review")
        test_path = os.path.join(path, "unlabeled.review")

        f = open(positive_path)
        line = f.readline()
        while line != "":
            train_docs.append(line)
            line = f.readline()
        f.close()

        f = open(negative_path)
        line = f.readline()
        while line != "":
            train_docs.append(line)
            line = f.readline()
        f.close()

        f = open(test_path)
        line = f.readline()
        while line != "":
            test_docs.append(line)
            line = f.readline()
        f.close()

    return train_docs, test_docs

In [30]:
def test_multinomial_nv_by_cat():
    for cat in CATEGORY_PATHS:
        print("=== Multinomial NV for:", cat, "===")
        folder_path = CATEGORY_PATHS[cat]
        train_docs, test_docs = get_docs_from_folder_paths([folder_path])
        train_corpus = Corpus(train_docs)
        test_corpus = Corpus(test_docs, train_corpus.getVocabulary())
        y_train = train_corpus.getDocuments()["label"]
        y_test = test_corpus.getDocuments()["label"]

        # Test with TFs
        print("TFs: ", end="")
        X_train = train_corpus.getTfs()
        X_test = test_corpus.getTfs()
        print(test_nv_classifier(X_train, y_train, X_test, y_test))

        # Test with TFIDFs
        print("TFIDFs: ", end="")
        X_train = train_corpus.getTfidfs()
        X_test = test_corpus.getTfidfs()
        print(test_nv_classifier(X_train, y_train, X_test, y_test))

In [31]:
test_multinomial_nv_by_cat()

=== Multinomial NV for: books ===
TFs: {'presition': 0.8158333333333333, 'recall': 0.8648409893992933, 'f1': 0.8396226415094339}
TFIDFs: {'presition': 0.827893175074184, 'recall': 0.8626325088339223, 'f1': 0.8449059052563271}
=== Multinomial NV for: dvd ===
TFs: {'presition': 0.7673860911270983, 'recall': 0.8854454897620365, 'f1': 0.8221993833504625}
TFIDFs: {'presition': 0.797514241325738, 'recall': 0.8522412838959601, 'f1': 0.8239700374531835}
=== Multinomial NV for: electronics ===
TFs: {'presition': 0.8196090996475489, 'recall': 0.8953447672383619, 'f1': 0.8558046169287387}
TFIDFs: {'presition': 0.8348961821835231, 'recall': 0.8725936296814841, 'f1': 0.8533287694677393}
=== Multinomial NV for: kitchen ===
TFs: {'presition': 0.866996699669967, 'recall': 0.8893026404874746, 'f1': 0.8780080213903744}
TFIDFs: {'presition': 0.8678045515394913, 'recall': 0.8777928232904536, 'f1': 0.8727701110737126}


In [33]:
from sklearn.linear_model import LogisticRegression
def test_lr_classifier(X_train: np.ndarray, y_train, X_test: np.ndarray, y_test) -> dict:
    model = LogisticRegression(penalty="l1", solver="liblinear")
    model.fit(X_train, y_train)
    y_predicted = model.predict(X_test)
    return calculate_metrics(y_test, y_predicted)

In [34]:
def test_lr_classifier_by_cat():
    for cat in CATEGORY_PATHS:
        print("=== Multinomial NV for:", cat, "===")
        folder_path = CATEGORY_PATHS[cat]
        train_docs, test_docs = get_docs_from_folder_paths([folder_path])
        train_corpus = Corpus(train_docs)
        test_corpus = Corpus(test_docs, train_corpus.getVocabulary())
        y_train = train_corpus.getDocuments()["label"]
        y_test = test_corpus.getDocuments()["label"]

        # Test with TFs
        print("TFs: ", end="")
        X_train = train_corpus.getTfs()
        X_test = test_corpus.getTfs()
        print(test_lr_classifier(X_train, y_train, X_test, y_test))

        # Test with TFIDFs
        print("TFIDFs: ", end="")
        X_train = train_corpus.getTfidfs()
        X_test = test_corpus.getTfidfs()
        print(test_lr_classifier(X_train, y_train, X_test, y_test))
        

In [36]:
test_lr_classifier_by_cat()

=== Multinomial NV for: books ===
TFs: {'presition': 0.8010182435299109, 'recall': 0.833922261484099, 'f1': 0.817139147370699}
TFIDFs: {'presition': 0.8293775190326914, 'recall': 0.8180212014134276, 'f1': 0.8236602179230599}
=== Multinomial NV for: dvd ===
TFs: {'presition': 0.7784522003034902, 'recall': 0.8516878804648589, 'f1': 0.8134249471458774}
TFIDFs: {'presition': 0.8061116965226555, 'recall': 0.8467072495849475, 'f1': 0.8259109311740892}
=== Multinomial NV for: electronics ===
TFs: {'presition': 0.8460754332313966, 'recall': 0.871543577178859, 'f1': 0.8586206896551724}
TFIDFs: {'presition': 0.8535750940814232, 'recall': 0.8732936646832342, 'f1': 0.8633217993079585}
=== Multinomial NV for: kitchen ===
TFs: {'presition': 0.8580923894981721, 'recall': 0.8740690589031821, 'f1': 0.8660070434345127}
TFIDFs: {'presition': 0.8710990502035278, 'recall': 0.8693297224102912, 'f1': 0.8702134869535751}


## 3. Construcción del modelo y métricas con un compilado de todas las categorías

In [ ]:
def test_nv_classifier_complete():
    print("=== TEST NV FOR FULL CORPUS ===")
    folder_paths = []
    for cat in CATEGORY_PATHS:
        folder_paths.append(CATEGORY_PATHS[cat])

    train_docs, test_docs = get_docs_from_folder_paths(folder_paths)
    train_corpus = Corpus(train_docs)
    test_corpus = Corpus(test_docs, train_corpus.getVocabulary())
    y_train = train_corpus.getDocuments()["label"]
    y_test = test_corpus.getDocuments()["label"]

    # Test with TFs
    print("TFs: ", end="")
    X_train = train_corpus.getTfs()
    X_test = test_corpus.getTfs()
    print(test_lr_classifier(X_train, y_train, X_test, y_test))

    # Test with TFIDFs
    print("TFIDFs: ", end="")
    X_train = train_corpus.getTfidfs()
    X_test = test_corpus.getTfidfs()
    print(test_lr_classifier(X_train, y_train, X_test, y_test))